In [1]:
import sys
sys.path.append('..')

from datetime import datetime, timedelta, date
from dateutil.relativedelta import relativedelta
from enum import Enum

from src.services import google_campaign_type_descriptive_saver as gcd
from src.database.session import SessionLocal
from src.pingers import *
from src.utils import *
from src import crud
from src.abc.descriptive import DescriptiveDF
db = SessionLocal()
end_date_plus_one = date(year=2023, month=8, day=1)
end_date = end_date_plus_one - timedelta(days=1)
start_date = end_date_plus_one - relativedelta(months=2)
shop_id = 16038

2023-08-29 09:49:39.539 | DEBUG    | src.database.session:<module>:17 - base: prod.cw6tznt1p9hq.us-east-1.rds.amazonaws.com


In [3]:
df = gcd.read_df(df_type=DescriptiveDF.main)

In [4]:
df.feature_value.unique()

array(['SEARCH', 'SHOPPING', 'SMART', 'PERFORMANCE_MAX', '.', 'DISPLAY',
       'UNKNOWN', 'DISCOVERY', 'VIDEO', 'LOCAL', 'MULTI_CHANNEL',
       'LOCAL_SERVICES'], dtype=object)

In [2]:
gcd.get_shop_descriptive_df(db=db, shop_id=16038, start_date=start_date, end_date=end_date)

2023-08-29 07:01:17.835 | DEBUG    | src.abc.descriptive:get_shop_descriptive_df:91 - 16038: +


KeyError: 'shop_id'

In [3]:
failed = gcd.read_df(df_type=DescriptiveDF.failed_shop_ids, end_date=end_date)
failed

,shop_id


In [2]:
gcd.create_and_save_main(db=db, end_date=end_date_plus_one, force_from_scratch=True)

100%|██████████| 2/2 [00:05<00:00,  2.90s/it]


In [4]:
main_df = gcd.read_df('main', end_date=end_date)
main_df

,Unnamed: 0,year_month,feature_value,spend_USD,impr,clicks,purch,purch_value_USD,n_ads,feature,shop_id
0,0,2021-08,DISPLAY,1121.326475,193990.0,779.0,24041.000000,91862.100000,2,campaign_type,16038
1,1,2021-08,SEARCH,1863.560000,17086.0,7850.0,234468.000000,867341.000000,5,campaign_type,16038
2,2,2021-08,SHOPPING,7051.145484,763241.0,10585.0,144939.000000,352955.800000,1,campaign_type,16038
3,3,2021-09,DISPLAY,1358.042882,188032.0,654.0,27187.934830,90420.213879,2,campaign_type,16038
4,4,2021-09,SEARCH,1057.680000,9191.0,4413.0,135261.000000,401353.350000,4,campaign_type,16038
...,...,...,...,...,...,...,...,...,...,...,...
168,19,2023-03,.,3908.292196,2094855.0,16698.0,123565.722646,496983.518763,3,fake_feature,44301396
169,20,2023-04,.,2768.585806,701866.0,10675.0,88834.946671,216962.645487,3,fake_feature,44301396
170,21,2023-05,.,4956.053326,3025271.0,19589.0,161732.953278,273136.693751,10,fake_feature,44301396
171,22,2023-06,.,2725.004547,648900.0,8611.0,54218.700834,86543.801833,25,fake_feature,44301396


In [5]:
gcd.create_and_save_summary(end_date=end_date)

2023-08-21 13:10:09.205 | DEBUG    | src.services.descriptive.descriptive_saver:create_and_save_summary:36 - ['year_month', 'feature', 'feature_value']


In [19]:
df = gcd.get_shop_df(db=db, shop_id=16038, start_date=start_date, end_date=end_date)
df[df.year_month == '2023-07'].sum()

ad_id                                  1874631945794
shop_id                                        64152
account_id                               10869124636
adgroup_id                              414152110626
campaign_type      PERFORMANCE_MAXSEARCHSEARCHSEARCH
spend                                    4436.698642
impr                                        498333.0
clicks                                       23112.0
purch                                  749964.483374
purch_value                           1849608.573213
year_month              2023-072023-072023-072023-07
spend_USD                                4436.698642
purch_value_USD                       1849608.573213
dtype: object

In [9]:
df = gcd.get_shop_descriptive_df(db=db, shop_id=44301396, start_date=start_date, end_date=end_date)
df

2023-08-21 13:14:08.304 | DEBUG    | src.services.descriptive.descriptive_saver:get_shop_descriptive_df:141 - 44301396: +


,year_month,feature_value,spend_USD,impr,clicks,purch,purch_value_USD,n_ads,feature,shop_id
0,2023-06,DISPLAY,18.800868,9877.0,44.0,204.446316,91.104097,3,campaign_type,44301396
1,2023-06,PERFORMANCE_MAX,2174.526064,529927.0,6511.0,46714.111817,73240.824643,1,campaign_type,44301396
2,2023-06,SEARCH,54.416000,1482.0,82.0,440.323608,1290.527162,16,campaign_type,44301396
3,2023-06,SHOPPING,35.419440,23752.0,71.0,477.700456,471.920607,4,campaign_type,44301396
4,2023-06,SMART,441.842176,83862.0,1903.0,6382.118637,11449.425325,1,campaign_type,44301396
5,2023-07,PERFORMANCE_MAX,3299.150298,251348.0,8435.0,95588.670031,199581.720814,1,campaign_type,44301396
6,2023-07,SEARCH,239.344855,2758.0,401.0,4345.432230,7107.486077,25,campaign_type,44301396
7,2023-07,SMART,580.298647,77551.0,2231.0,15440.719500,35302.726761,1,campaign_type,44301396
0,2023-06,.,2725.004547,648900.0,8611.0,54218.700834,86543.801833,25,fake_feature,44301396
1,2023-07,.,4118.793800,331657.0,11067.0,115374.821761,241991.933652,27,fake_feature,44301396


In [16]:
summary_df = gcd.read_df('summary', end_date=end_date)
summary_df[summary_df.feature == 'fake_feature']

,year_month,feature,feature_value,spend_USD,impr,clicks,purch,purch_value_USD,n_ads,n_shops,spend_USD_by_shop,impr_by_shop,clicks_by_shop,purch_by_shop,purch_value_USD_by_shop,n_ads_by_shop
4,2021-08,fake_feature,.,11601.815274,2231839.0,28059.0,4.862499e+05,1.482960e+06,13,2,1.0,1.0,1.0,1.0,1.0,1.0
9,2021-09,fake_feature,.,7864.217303,882102.0,22367.0,3.479052e+05,8.856725e+05,9,2,1.0,1.0,1.0,1.0,1.0,1.0
14,2021-10,fake_feature,.,11457.484180,2181830.0,51543.0,3.726019e+05,1.048597e+06,17,2,1.0,1.0,1.0,1.0,1.0,1.0
19,2021-11,fake_feature,.,11477.389758,1431005.0,26555.0,3.929199e+05,1.259096e+06,9,2,1.0,1.0,1.0,1.0,1.0,1.0
23,2021-12,fake_feature,.,7543.756629,1136000.0,18115.0,2.873648e+05,9.551879e+05,6,2,1.0,1.0,1.0,1.0,1.0,1.0
28,2022-01,fake_feature,.,13099.791853,1837049.0,26748.0,4.877808e+05,1.758468e+06,8,2,1.0,1.0,1.0,1.0,1.0,1.0
33,2022-02,fake_feature,.,15872.941859,2446701.0,68879.0,6.245447e+05,2.342524e+06,9,2,1.0,1.0,1.0,1.0,1.0,1.0
38,2022-03,fake_feature,.,33620.280242,3872826.0,158780.0,1.042799e+06,4.130284e+06,16,2,1.0,1.0,1.0,1.0,1.0,1.0
43,2022-04,fake_feature,.,15283.444683,2300716.0,46501.0,5.806642e+05,1.967822e+06,10,2,1.0,1.0,1.0,1.0,1.0,1.0
48,2022-05,fake_feature,.,10448.049678,1097078.0,58459.0,4.005616e+05,1.508737e+06,10,2,1.0,1.0,1.0,1.0,1.0,1.0


In [6]:
gcd.save_to_s3(db=db, end_date=end_date_plus_one, force_from_scratch=True)

/app/play/../src/services/descriptive/descriptive_saver.py:36: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  done_shop_ids = pd.Series([])
100%|██████████| 2/2 [00:06<00:00,  3.03s/it]


In [7]:
main_df = gcd.get_main_df(end_date=end_date)
main_df

spend_USD      impr   clicks  \
year_month feature       feature_value                                     
2023-06    campaign_type DISPLAY            18.776470    9877.0     44.0   
                         PERFORMANCE_MAX  5211.909738  932779.0  13375.0   
                         SEARCH            960.365413   18125.0  10568.0   
                         SHOPPING           35.373476   23752.0     71.0   
                         SMART             441.268802   83862.0   1903.0   
2023-07    campaign_type PERFORMANCE_MAX  6332.556304  725871.0  16609.0   
                         SEARCH           1638.045619   26568.0  15339.0   
                         SMART             579.545599   77551.0   2231.0   

                                                  purch  purch_value_USD  \
year_month feature       feature_value                                     
2023-06    campaign_type DISPLAY             204.446316     9.098587e+01   
                         PERFORMANCE_MAX  176268.778760     3.387327e+05   
                         SEARCH           371673.232087     8.470163e+05   
                         SHOPPING            477.700456     4.713082e+02   
                         SMART              6382.118637     1.143457e+04   
2023-07    campaign_type PERFORMANCE_MAX  273765.210971     6.325596e+05   
                         SEARCH           576133.374664     1.423470e+06   
                         SMART             15440.719500     3.525691e+04   

                                          n_ads  spend_USD_by_shop  \
year_month feature       feature_value                               
2023-06    campaign_type DISPLAY            3.0           0.006899   
                         PERFORMANCE_MAX    2.0           1.568398   
                         SEARCH            19.0           0.249561   
                         SHOPPING           4.0           0.012998   
                         SMART              1.0           0.162144   
2023-07    campaign_type PERFORMANCE_MAX    2.0           1.485672   
                         SEARCH            28.0           0.373438   
                         SMART              1.0           0.140890   

                                          impr_by_shop  clicks_by_shop  \
year_month feature       feature_value                                   
2023-06    campaign_type DISPLAY              0.015221        0.005110   
                         PERFORMANCE_MAX      1.776980        1.151745   
                         SEARCH               0.041958        0.613903   
                         SHOPPING             0.036603        0.008245   
                         SMART                0.129237        0.220996   
2023-07    campaign_type PERFORMANCE_MAX      1.710076        1.115845   
                         SEARCH               0.056095        0.682565   
                         SMART                0.233829        0.201590   

                                          purch_by_shop  \
year_month feature       feature_value                    
2023-06    campaign_type DISPLAY               0.003771   
                         PERFORMANCE_MAX       1.120289   
                         SEARCH                0.749419   
                         SHOPPING              0.008811   
                         SMART                 0.117711   
2023-07    campaign_type PERFORMANCE_MAX       1.066085   
                         SEARCH                0.800084   
                         SMART                 0.133831   

                                          purch_value_USD_by_shop  \
year_month feature       feature_value                              
2023-06    campaign_type DISPLAY                         0.001053   
                         PERFORMANCE_MAX                 1.085271   
                         SEARCH                          0.775927   
                         SHOPPING                        0.005453   
                         SMART                           0.132296   
2023-0

In [8]:
df_fw = gcd.get_shop_descriptive_df(db=db, shop_id=16038, end_date=end_date, start_date = start_date)
df_dr = gcd.get_shop_descriptive_df(db=db, shop_id=44301396, end_date=end_date, start_date = start_date)
df_test = df_fw.add(df_dr, fill_value=0)
df_test

spend_USD      impr   clicks  \
year_month feature       feature_value                                     
2023-06    campaign_type DISPLAY            18.776470    9877.0     44.0   
                         PERFORMANCE_MAX  5211.909738  932779.0  13375.0   
                         SEARCH            960.365413   18125.0  10568.0   
                         SHOPPING           35.373476   23752.0     71.0   
                         SMART             441.268802   83862.0   1903.0   
2023-07    campaign_type PERFORMANCE_MAX  6332.556304  725871.0  16609.0   
                         SEARCH           1638.045619   26568.0  15339.0   
                         SMART             579.545599   77551.0   2231.0   

                                                  purch  purch_value_USD  \
year_month feature       feature_value                                     
2023-06    campaign_type DISPLAY             204.446316     9.098587e+01   
                         PERFORMANCE_MAX  176268.778760     3.387327e+05   
                         SEARCH           371673.232087     8.470163e+05   
                         SHOPPING            477.700456     4.713082e+02   
                         SMART              6382.118637     1.143457e+04   
2023-07    campaign_type PERFORMANCE_MAX  273765.210971     6.325596e+05   
                         SEARCH           576133.374664     1.423470e+06   
                         SMART             15440.719500     3.525691e+04   

                                          n_ads  spend_USD_by_shop  \
year_month feature       feature_value                               
2023-06    campaign_type DISPLAY            3.0           0.006899   
                         PERFORMANCE_MAX    2.0           1.568398   
                         SEARCH            19.0           0.249561   
                         SHOPPING           4.0           0.012998   
                         SMART              1.0           0.162144   
2023-07    campaign_type PERFORMANCE_MAX    2.0           1.485672   
                         SEARCH            28.0           0.373438   
                         SMART              1.0           0.140890   

                                          impr_by_shop  clicks_by_shop  \
year_month feature       feature_value                                   
2023-06    campaign_type DISPLAY              0.015221        0.005110   
                         PERFORMANCE_MAX      1.776980        1.151745   
                         SEARCH               0.041958        0.613903   
                         SHOPPING             0.036603        0.008245   
                         SMART                0.129237        0.220996   
2023-07    campaign_type PERFORMANCE_MAX      1.710076        1.115845   
                         SEARCH               0.056095        0.682565   
                         SMART                0.233829        0.201590   

                                          purch_by_shop  \
year_month feature       feature_value                    
2023-06    campaign_type DISPLAY               0.003771   
                         PERFORMANCE_MAX       1.120289   
                         SEARCH                0.749419   
                         SHOPPING              0.008811   
                         SMART                 0.117711   
2023-07    campaign_type PERFORMANCE_MAX       1.066085   
                         SEARCH                0.800084   
                         SMART                 0.133831   

                                          purch_value_USD_by_shop  \
year_month feature       feature_value                              
2023-06    campaign_type DISPLAY                         0.001053   
                         PERFORMANCE_MAX                 1.085271   
                         SEARCH                          0.775927   
                         SHOPPING                        0.005453   
                         SMART                           0.132296   
2023-0

In [12]:
df_fw

spend_USD      impr   clicks  \
year_month feature       feature_value                                     
2023-06    campaign_type PERFORMANCE_MAX  3040.205536  402852.0   6864.0   
                         SEARCH            906.020029   16643.0  10486.0   
2023-07    campaign_type PERFORMANCE_MAX  3037.687282  474523.0   8174.0   
                         SEARCH           1399.011360   23810.0  14938.0   

                                                  purch  purch_value_USD  \
year_month feature       feature_value                                     
2023-06    campaign_type PERFORMANCE_MAX  129554.666943     2.655869e+05   
                         SEARCH           371232.908479     8.457275e+05   
2023-07    campaign_type PERFORMANCE_MAX  178176.540940     4.332369e+05   
                         SEARCH           571787.942434     1.416372e+06   

                                          n_ads  spend_USD_by_shop  \
year_month feature       feature_value                               
2023-06    campaign_type PERFORMANCE_MAX      1           0.770408   
                         SEARCH               3           0.229592   
2023-07    campaign_type PERFORMANCE_MAX      1           0.684673   
                         SEARCH               3           0.315327   

                                          impr_by_shop  clicks_by_shop  \
year_month feature       feature_value                                   
2023-06    campaign_type PERFORMANCE_MAX      0.960326        0.395620   
                         SEARCH               0.039674        0.604380   
2023-07    campaign_type PERFORMANCE_MAX      0.952221        0.353669   
                         SEARCH               0.047779        0.646331   

                                          purch_by_shop  \
year_month feature       feature_value                    
2023-06    campaign_type PERFORMANCE_MAX       0.258702   
                         SEARCH                0.741298   
2023-07    campaign_type PERFORMANCE_MAX       0.237580   
                         SEARCH                0.762420   

                                          purch_value_USD_by_shop  \
year_month feature       feature_value                              
2023-06    campaign_type PERFORMANCE_MAX                 0.238984   
                         SEARCH                          0.761016   
2023-07    campaign_type PERFORMANCE_MAX                 0.234232   
                         SEARCH                          0.765768   

                                          n_ads_by_shop  n_shops  
year_month feature       feature_value                            
2023-06    campaign_type PERFORMANCE_MAX           0.25        1  
                         SEARCH                    0.75        1  
2023-07    campaign_type PERFORMANCE_MAX           0.25        1  
                         SEARCH                    0.75        1

In [11]:
main_df.subtract(df_test).max()

spend_USD                  0.000000e+00
impr                       0.000000e+00
clicks                     0.000000e+00
purch                      0.000000e+00
purch_value_USD            5.684342e-14
n_ads                      0.000000e+00
spend_USD_by_shop          0.000000e+00
impr_by_shop               2.220446e-16
clicks_by_shop             0.000000e+00
purch_by_shop              0.000000e+00
purch_value_USD_by_shop    0.000000e+00
n_ads_by_shop              0.000000e+00
n_shops                    0.000000e+00
dtype: float64

In [ ]:
df_dr

spend_USD       impr  \
year_month feature              feature_value                            
2023-06    weasel_words         False           6049.274407  1310240.0   
2023-07    weasel_words         False          20051.149057  3588629.0   
2023-06    link                 False           6049.274407  1310240.0   
2023-07    link                 False          20051.149057  3588629.0   
2023-06    free_shipping        False           6049.274407  1310240.0   
2023-07    free_shipping        False          20051.149057  3588629.0   
2023-06    discount             False           6049.274407  1310240.0   
2023-07    discount             False          10635.536109  1890606.0   
                                True            9415.612948  1698023.0   
2023-06    urgency              False           3238.773401   710777.0   
                                True            2810.501006   599463.0   
2023-07    urgency              False          12331.324074  2162588.0   
                                True            7719.824983  1426041.0   
2023-06    price                False           6049.274407  1310240.0   
2023-07    price                False          20051.149057  3588629.0   
2023-06    cta                  False           3238.773401   710777.0   
                                True            2810.501006   599463.0   
2023-07    cta                  False          14217.503899  2524886.0   
                                True            5833.645158  1063743.0   
2023-06    fact_words           False           4623.339767   990224.0   
                                True            1425.934639   320016.0   
2023-07    fact_words           False          20051.149057  3588629.0   
2023-06    hashtag              False           6049.274407  1310240.0   
2023-07    hashtag              False          20051.149057  3588629.0   
2023-06    starts_with_question False           6049.274407  1310240.0   
2023-07    starts_with_question False          20051.149057  3588629.0   
2023-06    user_addressing      True            6049.274407  1310240.0   
2023-07    user_addressing      True           20051.149057  3588629.0   
2023-06    emoji                False           3846.804758   819106.0   
                                True            2202.469648   491134.0   
2023-07    emoji                False          14428.168659  2571570.0   
                                True            5622.980398  1017059.0   
2023-06    percentage           False           4623.339767   990224.0   
                                True            1425.934639   320016.0   
2023-07    percentage           False          10635.536109  1890606.0   
                                True            9415.612948  1698023.0   

                                                clicks   purch  purch_value  \
year_month feature              feature_value                                 
2023-06    weasel_words         False           6475.0   155.0     18967.06   
2023-07    weasel_words         False          23924.0  1206.0    108585.80   
2023-06    link                 False           6475.0   155.0     18967.06   
2023-07    link                 False          23924.0  1206.0    108585.80   
2023-06    free_shipping        False           6475.0   155.0     18967.06   
2023-07    free_shipping        False          23924.0  1206.0    108585.80   
2023-06    discount             False           6475.0   155.0     18967.06   
2023-07    discount             False           9234.0   353.0     37790.57   
                                True           14690.0   853.0     70795.23   
2023-06    urgency              False           3765.0    76.0      8462.44   
                                True            2710.0    79.0     10504.62   
2023-07    urgency              False          15726.0   782.0     67048.52   
                                True            8198.0   424.0     41537.28   
2023-06    price                False           6475